In [1]:
# Step1: Generate each_contest_before_csv
import xml.etree.ElementTree as ET
import pandas as pd  

# Input will be the card_ID 
cols = ["車號","姓名", "rfid", "車隊", "車型", "日期"]
rows = []

#Parse XML file
tree = ET.parse('race_cars_info_tester.xml')
root = tree.getroot()
for elem in root:
    
    name = elem.find("姓名").text
    car_label = elem.find("車號").text
    rfid = elem.find("rfid").text
    car_team = elem.find("車隊").text
    car_model = elem.find("車型").text
    date = elem.find("日期").text 

    rows.append({"姓名": name, 
              "車號": car_label,
              "rfid": rfid,
              "車隊": car_team,
              "車型": car_model,
              "日期": date})

df = pd.DataFrame(rows, columns = cols) 
  
# # write dataframe to csv
df.to_csv('each_contest_before.csv',index=False)

key = cardId = string = "1014E3_op"       --> value = 車號


In [2]:
df

,車號,姓名,rfid,車隊,車型,日期
0,1,Steven,1014E9,team 1,BMW M4,2022年6月15日
1,2,Michael,10155D,team 2,McLaren 720S,2022年6月15日


In [3]:
#1. Insert each personName with corresponding cardID
# Need All people's cardIDs!!!


import math
cardIDMap = {}
bestScoreMap = {}
rfidList = df['rfid'].tolist()
idList = df['車號'].tolist()


for i, rfid in enumerate(rfidList):
    cardIDMap[rfid] = idList[i]
    bestScoreMap[rfid] = math.inf

In [4]:
print(cardIDMap)

{'1014E9': '1', '10155D': '2'}


In [5]:
print(bestScoreMap)

{'1014E9': inf, '10155D': inf}


# 模擬三張不同的卡號 給予六圈的紀錄（修改前面10 個字數的絕對時間點）


----------------- 1014E3_op（卡號 ＃1) ----------------- 此卡號給steven


1651545100 472058 2022/05/03 Tue - 10:24:00.472


1651545200 686995 2022/05/03 Tue - 10:32:09.686


1651545300 408053 2022/05/03 Tue - 10:34:14.408


1651545400 79984 2022/05/03 Tue - 10:38:14.079


1651545500 802198 2022/05/03 Tue - 10:42:09.802


1651545600 426224 2022/05/03 Tue - 10:43:05.426





----------------- 1014E9_op（卡號 ＃2)----------------- 此卡號給Albert


1651545100 472058 2022/05/03 Tue - 10:33:00.472


1651545300 686995 2022/05/03 Tue - 10:35:09.686


1651545450 408053 2022/05/03 Tue - 10:37:14.408


1651545500 79984 2022/05/03 Tue - 10:39:14.079


1651545670 802198 2022/05/03 Tue - 10:41:10.802


1651545850 426224 2022/05/03 Tue - 10:43:05.426



----------------- 10155D_op（卡號 ＃3) ----------------- 此車號給Michael


1651545180 472058 2022/05/03 Tue - 10:33:00.472

1651545309 686995 2022/05/03 Tue - 10:35:09.686

1651545509 686995 2022/05/03 Tue - 10:37:09.686

1651545559 686995 2022/05/03 Tue - 10:39:49.236


In [6]:
# Input = HH:MM:SS.SSS
# Output = miliSecond Timestamp

# 1 hr = 60 * 60 * 1000 = 3_600_000 
# 1 min = 60 * 1000 = 60_000  
# 1 sec = 1000 = 1_000  

from datetime import timedelta
def convertToTimeStamp(timeStr):
    h, m, s = map(float, timeStr.split(':'))
    res = timedelta(hours=h, minutes=m, seconds=s).total_seconds() * 1000
    return int(res)

In [7]:
convertToTimeStamp("16:06:33.021")

57993021

In [8]:
convertToTimeStamp("16:08:58.424")

58138424

In [9]:
# Input = mm
# Output = nn:sec.mm
def formattedTime(timeStamp):
    
    msPerMin = 60*1000

    minutes = math.trunc(timeStamp/(msPerMin))
    seconds = math.trunc((timeStamp - (minutes * msPerMin)) / 1000); 
    miliSeconds = timeStamp % 1000; 

    minutes = str(minutes).rjust(2,"0")
    seconds = str(seconds).rjust(2,"0")
    miliSeconds = str(miliSeconds).rjust(3,"0")
            
    return (minutes + ":" + seconds + "." + miliSeconds)

In [10]:
formattedTime(58138424-57993021)

'02:25.403'

In [11]:
def readCSVAndparseOpFiles():

    df = pd.read_csv ('each_contest_before.csv')
    df['車競賽'] = 1

    import glob, os
    
    # Please change path to be the current directory. 
    os.chdir("/Users/stevenkan/Documents/Project/Race_Car_Stream")


    for cardIDfileName in glob.glob("*op"):

        time = open(cardIDfileName).readlines()
        cardID = cardIDfileName[:6]
        
        # Skip If cardID map doesn't contain the given key
        if cardID not in cardIDMap:
            print(cardID + "_op 不在此車競賽! 不做任何分析")
            continue
        
        personID = cardIDMap[cardID]

        # Extract each timestamp in each line. 
        i = 0
        prev = 0
        for line in time:
            timestamp = convertToTimeStamp(line[-13:])
            
#             print("timestamp: "+ str(timestamp))

            index = df.index
            new_val = df['車號'] == int(personID)
            index = index[new_val]

            
            timeTravel = timestamp - prev if prev != 0 else math.inf
            bestScoreMap[cardID] = min(bestScoreMap[cardID], timeTravel)
            
            if not math.isinf(timeTravel):
                
                formattedTimeTravel = formattedTime(timeTravel)

                if(i == 1):
                    df.loc[index, '第一圈秒數'] = formattedTimeTravel
                elif(i == 2):
                    df.loc[index, '第二圈秒數'] = formattedTimeTravel
                elif(i == 3):
                    df.loc[index, '第三圈秒數'] = formattedTimeTravel
                elif(i == 4):
                    df.loc[index, '第四圈秒數'] = formattedTimeTravel
                elif(i == 5):
                    df.loc[index, '第五圈秒數'] = formattedTimeTravel
                elif(i == 6):
                    df.loc[index, '第六圈秒數'] = formattedTimeTravel
                elif(i == 7):
                    df.loc[index, '第七圈秒數'] = formattedTimeTravel
                elif(i == 8):
                    df.loc[index, '第八圈秒數'] = formattedTimeTravel
                elif(i == 9):
                    df.loc[index, '第九圈秒數'] = formattedTimeTravel
                elif(i == 10):
                    df.loc[index, '第十圈秒數'] = formattedTimeTravel

            i = i + 1
            prev = timestamp


        print("--------------End of " + cardIDfileName + "---------------")
    return df

In [12]:
def appendBestScore(df):
    # k is rfid value and v is best score 
    for k,v in bestScoreMap.items():
        # find the index of the tuple 
        i = df[df['車號'] == int(cardIDMap[k])].index.values[0]
        # update the cell
        df.loc[i,'最佳圈秒數'] = formattedTime(bestScoreMap[k]) if not math.isinf(bestScoreMap[k]) else "N/A"
        # rank. -1 means that the rank is currently unavailable
        df.loc[i,'排名'] = bestScoreMap[k] if not math.isinf(bestScoreMap[k]) else math.inf
        df = df.sort_values('排名', ascending= True)
        
    return df

In [13]:
# How to resolve the problem when partial op files are generated, but some don't? 
# How to adjust the rank accordingly? 



# bestScores =   5  7   9  10 10
#   rank = 0     1  2   3  4   4
#   prev = -inf  5  7   9  10  10

#   rankList  = { 1, 1}


# Edge Case #1 = 1234, 1234, 1235, inf, inf 

#   rankList =  {  1,    1,    2,   3,   3}


# Edge Case #0 = inf, inf, inf, inf, inf

#   rankList =  {  1,    1,  1,   1,   1}

def appendRank(df):
    bestScoreList = list(df['排名'])

    rankList = []
    
    rank = 0
    prev = -math.inf
    
    
    for val in bestScoreList:
        if(val > prev):
            rank = rank + 1
            
        rankList.append(rank)
        prev = val
    
    df = df.drop(['排名'], axis=1)
    df['排名'] = rankList

    return df


In [14]:
df

,車號,姓名,rfid,車隊,車型,日期
0,1,Steven,1014E9,team 1,BMW M4,2022年6月15日
1,2,Michael,10155D,team 2,McLaren 720S,2022年6月15日


In [15]:
# Step3: Convert csv to xml
import csv

def writeToXML():
    csvFile = 'each_contest_after.csv'
    xmlFile = 'result.xml'

    csvData = csv.reader(open(csvFile))
    xmlData = open(xmlFile, 'w')
    xmlData.write('<?xml version="1.0"?>' + "\n")
    # there must be only one top-level tag
    xmlData.write('<csv_data>' + "\n")

    rowNum = 0
    for row in csvData:
        if rowNum == 0:
            tags = row
            # replace spaces w/ underscores in tag names
            for i in range(len(tags)):
                tags[i] = tags[i].replace(' ', '_')
        else: 
            xmlData.write('<row>' + "\n")
            for i in range(len(tags)):
                xmlData.write('    ' + '<' + tags[i] + '>' \
                              + row[i] + '</' + tags[i] + '>' + "\n")
            xmlData.write('</row>' + "\n")

        rowNum +=1

    xmlData.write('</csv_data>' + "\n")
    xmlData.close()

In [16]:
import time

apprx_finish_second = 5

start_time = time.time()
while(True):
    current_time = time.time()
    elapsed_time = current_time - start_time
    
    if elapsed_time > apprx_finish_second:
        break    
    
    df = readCSVAndparseOpFiles()
    df = appendBestScore(df)
    df = appendRank(df)
    df.to_csv('each_contest_after.csv', index=False)
    print(df)
    writeToXML()
    

10154D_op 不在此車競賽! 不做任何分析
--------------End of 10155D_op---------------
--------------End of 1014E9_op---------------
   車號       姓名    rfid      車隊            車型          日期  車競賽      第一圈秒數  \
1   2  Michael  10155D  team 2  McLaren 720S  2022年6月15日    1  00:01.000   
0   1   Steven  1014E9  team 1        BMW M4  2022年6月15日    1  00:01.000   

       第二圈秒數      第三圈秒數      第四圈秒數      第五圈秒數      第六圈秒數      第七圈秒數  \
1  02:00.000  04:10.000  00:50.000  01:30.840  01:29.160  05:20.060   
0        NaN        NaN        NaN        NaN        NaN        NaN   

       第八圈秒數      第九圈秒數      第十圈秒數      最佳圈秒數  排名  
1  01:00.000  00:00.001  00:59.999  00:00.001   1  
0        NaN        NaN        NaN  00:01.000   2  
10154D_op 不在此車競賽! 不做任何分析
--------------End of 10155D_op---------------
--------------End of 1014E9_op---------------
   車號       姓名    rfid      車隊            車型          日期  車競賽      第一圈秒數  \
1   2  Michael  10155D  team 2  McLaren 720S  2022年6月15日    1  00:01.000   
0   1   Steven 

10154D_op 不在此車競賽! 不做任何分析
--------------End of 10155D_op---------------
--------------End of 1014E9_op---------------
   車號       姓名    rfid      車隊            車型          日期  車競賽      第一圈秒數  \
1   2  Michael  10155D  team 2  McLaren 720S  2022年6月15日    1  00:01.000   
0   1   Steven  1014E9  team 1        BMW M4  2022年6月15日    1  00:01.000   

       第二圈秒數      第三圈秒數      第四圈秒數      第五圈秒數      第六圈秒數      第七圈秒數  \
1  02:00.000  04:10.000  00:50.000  01:30.840  01:29.160  05:20.060   
0        NaN        NaN        NaN        NaN        NaN        NaN   

       第八圈秒數      第九圈秒數      第十圈秒數      最佳圈秒數  排名  
1  01:00.000  00:00.001  00:59.999  00:00.001   1  
0        NaN        NaN        NaN  00:01.000   2  
10154D_op 不在此車競賽! 不做任何分析
--------------End of 10155D_op---------------
--------------End of 1014E9_op---------------
   車號       姓名    rfid      車隊            車型          日期  車競賽      第一圈秒數  \
1   2  Michael  10155D  team 2  McLaren 720S  2022年6月15日    1  00:01.000   
0   1   Steven 

   車號       姓名    rfid      車隊            車型          日期  車競賽      第一圈秒數  \
1   2  Michael  10155D  team 2  McLaren 720S  2022年6月15日    1  00:01.000   
0   1   Steven  1014E9  team 1        BMW M4  2022年6月15日    1  00:01.000   

       第二圈秒數      第三圈秒數      第四圈秒數      第五圈秒數      第六圈秒數      第七圈秒數  \
1  02:00.000  04:10.000  00:50.000  01:30.840  01:29.160  05:20.060   
0        NaN        NaN        NaN        NaN        NaN        NaN   

       第八圈秒數      第九圈秒數      第十圈秒數      最佳圈秒數  排名  
1  01:00.000  00:00.001  00:59.999  00:00.001   1  
0        NaN        NaN        NaN  00:01.000   2  
10154D_op 不在此車競賽! 不做任何分析
--------------End of 10155D_op---------------
--------------End of 1014E9_op---------------
   車號       姓名    rfid      車隊            車型          日期  車競賽      第一圈秒數  \
1   2  Michael  10155D  team 2  McLaren 720S  2022年6月15日    1  00:01.000   
0   1   Steven  1014E9  team 1        BMW M4  2022年6月15日    1  00:01.000   

       第二圈秒數      第三圈秒數      第四圈秒數      第五圈秒數      第六圈秒

   車號       姓名    rfid      車隊            車型          日期  車競賽      第一圈秒數  \
1   2  Michael  10155D  team 2  McLaren 720S  2022年6月15日    1  00:01.000   
0   1   Steven  1014E9  team 1        BMW M4  2022年6月15日    1  00:01.000   

       第二圈秒數      第三圈秒數      第四圈秒數      第五圈秒數      第六圈秒數      第七圈秒數  \
1  02:00.000  04:10.000  00:50.000  01:30.840  01:29.160  05:20.060   
0        NaN        NaN        NaN        NaN        NaN        NaN   

       第八圈秒數      第九圈秒數      第十圈秒數      最佳圈秒數  排名  
1  01:00.000  00:00.001  00:59.999  00:00.001   1  
0        NaN        NaN        NaN  00:01.000   2  
10154D_op 不在此車競賽! 不做任何分析
--------------End of 10155D_op---------------
--------------End of 1014E9_op---------------
   車號       姓名    rfid      車隊            車型          日期  車競賽      第一圈秒數  \
1   2  Michael  10155D  team 2  McLaren 720S  2022年6月15日    1  00:01.000   
0   1   Steven  1014E9  team 1        BMW M4  2022年6月15日    1  00:01.000   

       第二圈秒數      第三圈秒數      第四圈秒數      第五圈秒數      第六圈秒

--------------End of 10155D_op---------------
--------------End of 1014E9_op---------------
   車號       姓名    rfid      車隊            車型          日期  車競賽      第一圈秒數  \
1   2  Michael  10155D  team 2  McLaren 720S  2022年6月15日    1  00:01.000   
0   1   Steven  1014E9  team 1        BMW M4  2022年6月15日    1  00:01.000   

       第二圈秒數      第三圈秒數      第四圈秒數      第五圈秒數      第六圈秒數      第七圈秒數  \
1  02:00.000  04:10.000  00:50.000  01:30.840  01:29.160  05:20.060   
0        NaN        NaN        NaN        NaN        NaN        NaN   

       第八圈秒數      第九圈秒數      第十圈秒數      最佳圈秒數  排名  
1  01:00.000  00:00.001  00:59.999  00:00.001   1  
0        NaN        NaN        NaN  00:01.000   2  
10154D_op 不在此車競賽! 不做任何分析
--------------End of 10155D_op---------------
--------------End of 1014E9_op---------------
   車號       姓名    rfid      車隊            車型          日期  車競賽      第一圈秒數  \
1   2  Michael  10155D  team 2  McLaren 720S  2022年6月15日    1  00:01.000   
0   1   Steven  1014E9  team 1        BM

...

--------------End of 10155D_op---------------
--------------End of 1014E9_op---------------
   車號       姓名    rfid      車隊            車型          日期  車競賽      第一圈秒數  \
1   2  Michael  10155D  team 2  McLaren 720S  2022年6月15日    1  00:01.000   
0   1   Steven  1014E9  team 1        BMW M4  2022年6月15日    1  00:01.000   

       第二圈秒數      第三圈秒數      第四圈秒數      第五圈秒數      第六圈秒數      第七圈秒數  \
1  02:00.000  04:10.000  00:50.000  01:30.840  01:29.160  05:20.060   
0        NaN        NaN        NaN        NaN        NaN        NaN   

       第八圈秒數      第九圈秒數      第十圈秒數      最佳圈秒數  排名  
1  01:00.000  00:00.001  00:59.999  00:00.001   1  
0        NaN        NaN        NaN  00:01.000   2  
10154D_op 不在此車競賽! 不做任何分析
--------------End of 10155D_op---------------
--------------End of 1014E9_op---------------
   車號       姓名    rfid      車隊            車型          日期  車競賽      第一圈秒數  \
1   2  Michael  10155D  team 2  McLaren 720S  2022年6月15日    1  00:01.000   
0   1   Steven  1014E9  team 1        BM

0        NaN        NaN        NaN  00:01.000   2  
10154D_op 不在此車競賽! 不做任何分析
--------------End of 10155D_op---------------
--------------End of 1014E9_op---------------
   車號       姓名    rfid      車隊            車型          日期  車競賽      第一圈秒數  \
1   2  Michael  10155D  team 2  McLaren 720S  2022年6月15日    1  00:01.000   
0   1   Steven  1014E9  team 1        BMW M4  2022年6月15日    1  00:01.000   

       第二圈秒數      第三圈秒數      第四圈秒數      第五圈秒數      第六圈秒數      第七圈秒數  \
1  02:00.000  04:10.000  00:50.000  01:30.840  01:29.160  05:20.060   
0        NaN        NaN        NaN        NaN        NaN        NaN   

       第八圈秒數      第九圈秒數      第十圈秒數      最佳圈秒數  排名  
1  01:00.000  00:00.001  00:59.999  00:00.001   1  
0        NaN        NaN        NaN  00:01.000   2  
10154D_op 不在此車競賽! 不做任何分析
--------------End of 10155D_op---------------
--------------End of 1014E9_op---------------
   車號       姓名    rfid      車隊            車型          日期  車競賽      第一圈秒數  \
1   2  Michael  10155D  team 2  McLaren

   車號       姓名    rfid      車隊            車型          日期  車競賽      第一圈秒數  \
1   2  Michael  10155D  team 2  McLaren 720S  2022年6月15日    1  00:01.000   
0   1   Steven  1014E9  team 1        BMW M4  2022年6月15日    1  00:01.000   

       第二圈秒數      第三圈秒數      第四圈秒數      第五圈秒數      第六圈秒數      第七圈秒數  \
1  02:00.000  04:10.000  00:50.000  01:30.840  01:29.160  05:20.060   
0        NaN        NaN        NaN        NaN        NaN        NaN   

       第八圈秒數      第九圈秒數      第十圈秒數      最佳圈秒數  排名  
1  01:00.000  00:00.001  00:59.999  00:00.001   1  
0        NaN        NaN        NaN  00:01.000   2  
10154D_op 不在此車競賽! 不做任何分析
--------------End of 10155D_op---------------
--------------End of 1014E9_op---------------
   車號       姓名    rfid      車隊            車型          日期  車競賽      第一圈秒數  \
1   2  Michael  10155D  team 2  McLaren 720S  2022年6月15日    1  00:01.000   
0   1   Steven  1014E9  team 1        BMW M4  2022年6月15日    1  00:01.000   

       第二圈秒數      第三圈秒數      第四圈秒數      第五圈秒數      第六圈秒

--------------End of 10155D_op---------------
--------------End of 1014E9_op---------------
   車號       姓名    rfid      車隊            車型          日期  車競賽      第一圈秒數  \
1   2  Michael  10155D  team 2  McLaren 720S  2022年6月15日    1  00:01.000   
0   1   Steven  1014E9  team 1        BMW M4  2022年6月15日    1  00:01.000   

       第二圈秒數      第三圈秒數      第四圈秒數      第五圈秒數      第六圈秒數      第七圈秒數  \
1  02:00.000  04:10.000  00:50.000  01:30.840  01:29.160  05:20.060   
0        NaN        NaN        NaN        NaN        NaN        NaN   

       第八圈秒數      第九圈秒數      第十圈秒數      最佳圈秒數  排名  
1  01:00.000  00:00.001  00:59.999  00:00.001   1  
0        NaN        NaN        NaN  00:01.000   2  
10154D_op 不在此車競賽! 不做任何分析
--------------End of 10155D_op---------------
--------------End of 1014E9_op---------------
   車號       姓名    rfid      車隊            車型          日期  車競賽      第一圈秒數  \
1   2  Michael  10155D  team 2  McLaren 720S  2022年6月15日    1  00:01.000   
0   1   Steven  1014E9  team 1        BM

--------------End of 10155D_op---------------
--------------End of 1014E9_op---------------
   車號       姓名    rfid      車隊            車型          日期  車競賽      第一圈秒數  \
1   2  Michael  10155D  team 2  McLaren 720S  2022年6月15日    1  00:01.000   
0   1   Steven  1014E9  team 1        BMW M4  2022年6月15日    1  00:01.000   

       第二圈秒數      第三圈秒數      第四圈秒數      第五圈秒數      第六圈秒數      第七圈秒數  \
1  02:00.000  04:10.000  00:50.000  01:30.840  01:29.160  05:20.060   
0        NaN        NaN        NaN        NaN        NaN        NaN   

       第八圈秒數      第九圈秒數      第十圈秒數      最佳圈秒數  排名  
1  01:00.000  00:00.001  00:59.999  00:00.001   1  
0        NaN        NaN        NaN  00:01.000   2  
10154D_op 不在此車競賽! 不做任何分析
--------------End of 10155D_op---------------
--------------End of 1014E9_op---------------
   車號       姓名    rfid      車隊            車型          日期  車競賽      第一圈秒數  \
1   2  Michael  10155D  team 2  McLaren 720S  2022年6月15日    1  00:01.000   
0   1   Steven  1014E9  team 1        BM